# 파이프라인 만들기

Azure ML SDK를 사용해 스크립트 기반 실험을 실행하면 데이터를 수집하고 모델을 학습시킨 다음 개별적으로 등록하는 데 필요한 여러 단계를 수행할 수 있습니다. 그러나 엔터프라이즈 환경에서는 보통 기계 학습 솔루션을 빌드하려면 수행해야 하는 개별 단계 순서를 *파이프라인*에 캡슐화합니다. 이 파이프라인은 사용자의 요청 시 컴퓨팅 대상 하나 이상에서 실행하거나, 자동화된 빌드 프로세스에서 실행하거나 일정에 따라 실행할 수 있습니다.

이 Notebook에서는 이러한 모든 요소를 취합하여 데이터를 전처리한 다음 모델 학습과 등록을 진행하는 간단한 파이프라인을 만들어 보겠습니다.

## Azure Machine Learning SDK 설치

Azure Machine Learning SDK는 자주 업데이트됩니다. 다음 셀을 실행하여 최신 릴리스로 업그레이드하고 Notebook 위젯을 지원하는 추가 패키지를 다운로드하세요.

In [ ]:
!pip install --upgrade azureml-sdk azureml-widgets

## 작업 영역에 연결합니다.

최신 버전의 SDK를 설치했으므로 작업 영역에 연결할 수 있습니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증을 하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 데이터 준비

파이프라인에서는 당뇨병 환자의 세부 정보가 포함된 데이터 세트를 사용합니다. 아래 셀을 실행하여 이 데이터 세트를 만듭니다. 이전에 데이터 세트를 만든 경우, 코드가 기존 버전을 찾습니다.

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # /data에서 당뇨병 CSV 파일 업로드
                        target_path='diabetes-data/', # 데이터 저장소의 폴더 경로에 해당 파일 저장
                        overwrite=True, # 이름이 같은 기존 파일 바꾸기
                        show_progress=True)

    #데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # 테이블 형식 데이터 세트 등록
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## 파이프라인 단계용 스크립트 만들기

파이프라인은 *단계* 하나 이상으로 구성됩니다. 이러한 단계는 Python 스크립트일 수도 있고, 특정 위치 간에 데이터를 복사하는 데이터 전송 단계 등의 특수 단계일 수도 있습니다.  이 연습에서는 Python 스크립트 단계 2개가 포함된 간단한 파이프라인을 작성합니다. 한 단계에서는 학습 데이터를 전처리하며, 다른 단계에서는 전처리된 데이터를 사용하여 모델 학습과 등록을 진행합니다.

먼저 파이프라인 단계에서 사용할 스크립트 파일용 폴더를 만듭니다.

In [ ]:
import os
# 파이프라인 단계 파일용 폴더 만들기
experiment_folder = 'diabetes_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

이제 첫 번째 스크립트를 작성합니다. 이 스크립트는 당뇨병 데이터 세트의 데이터를 읽은 후 간단한 전처리 작업을 적용해 데이터가 누락된 행을 제거하고 숫자 특징을 크기가 비슷하도록 정규화합니다.

이 스크립트에 포함된 **--prepped-data** 인수는 결과 데이터를 저장해야 하는 폴더를 참조합니다.

In [ ]:
%%writefile $experiment_folder/prep_diabetes.py
# 라이브러리 가져오기
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# 매개 변수 가져오기
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 데이터 로드(입력 데이터 세트로 전달됨)
print("Loading Data...")
diabetes = run.input_datasets['raw_data'].to_pandas_dataframe()

# 원시 행 수 로깅
row_count = (len(diabetes))
run.log('raw_rows', row_count)

# Null 제거
diabetes = diabetes.dropna()

# 숫자 열 정규화
scaler = MinMaxScaler()
num_cols = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree']
diabetes[num_cols] = scaler.fit_transform(diabetes[num_cols])

# 처리된 행 로깅
row_count = (len(diabetes))
run.log('processed_rows', row_count)

# 준비된 데이터 저장
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
diabetes.to_csv(save_path, index=False, header=True)

# 실행 종료
run.complete()

이제 두 번째 단계(모델 학습)용 스크립트를 만들 수 있습니다. 이 스크립트에 포함된 **--training-folder** 인수는 이전 단계에서 준비한 데이터가 저장된 폴더를 참조합니다.

In [ ]:
%%writefile $experiment_folder/train_diabetes.py
# 라이브러리 가져오기
from azureml.core import Run, Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# 매개 변수 가져오기
parser = argparse.ArgumentParser()
parser.add_argument("--training-folder", type=str, dest='training_folder', help='training data folder')
args = parser.parse_args()
training_folder = args.training_folder

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 학습 폴더에 준비된 데이터 파일 로드
print("Loading Data...")
file_path = os.path.join(training_folder,'data.csv')
diabetes = pd.read_csv(file_path)

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model...')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# ROC 곡선 그리기
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# 대각선 50% 선 그리기
plt.plot([0, 1], [0, 1], 'k--')
# 모델의 FPR 및 TPR 그리기
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# outputs 폴더에 학습된 모델 저장
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'diabetes_model.pkl')
joblib.dump(value=model, filename=model_file)

# 모델 등록
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'diabetes_model',
               tags={'Training context':'Pipeline'},
               properties={'AUC': np.float(auc), 'Accuracy': np.float(acc)})


run.complete()

## 파이프라인 단계용 컴퓨팅 환경 준비

이 연습에서는 두 단계에 같은 컴퓨팅을 사용하지만 각 단계는 독립적으로 실행됩니다. 따라서 해당하는 경우 각 단계에 서로 다른 컴퓨팅 컨텍스트를 지정할 수 있습니다.

먼저, 이전 랩에서 만든 컴퓨팅 대상을 가져옵니다. 없는 경우에는 생성됩니다.

> **중요**: 계산 클러스터를 실행하기 전에 *your-compute-cluster*를 아래 코드에서 컴퓨팅 클러스터의 이름으로 변경하세요! 클러스터 이름은 길이가 2~16자 사이의 전역으로 고유한 이름이어야 합니다. 유효한 문자는 문자, 숫자 및 문자입니다.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "dp100-cluster"

try:
    # 기존 컴퓨팅 대상 확인
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # 아직 존재하지 않는 경우, 생성합니다.
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

컴퓨팅에는 필요한 패키지 종속성이 설치된 Python 환경이 필요하므로 실행 구성을 만들어야 합니다.

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# 실험용 Python 환경 만들기
diabetes_env = Environment("diabetes-pipeline-env")
diabetes_env.python.user_managed_dependencies = False # Azure ML의 종속성 관리 허용
diabetes_env.docker.enabled = True # Docker 컨테이너 사용

# 패키지 종속성 집합 만들기
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib','pandas','pip'],
                                             pip_packages=['azureml-defaults','azureml-dataprep[pandas]','pyarrow'])

# 환경에 종속성 추가
diabetes_env.python.conda_dependencies = diabetes_packages

# 환경 등록 
diabetes_env.register(workspace=ws)
registered_env = Environment.get(ws, 'diabetes-pipeline-env')

# 파이프라인용 새 runconfig 개체 만들기
pipeline_run_config = RunConfiguration()

# 위에서 만든 컴퓨팅을 사용합니다. 
pipeline_run_config.target = pipeline_cluster

# 실행 구성에 환경 할당
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

## 파이프라인 작성 및 실행

이제 파이프라인을 만들고 실행할 수 있습니다.

먼저 파이프라인용 단계와 파이프라인 간에 전달해야 하는 데이터 참조를 정의해야 합니다. 이 연습의 첫 번째 단계는 두 번째 단계에서 읽을 수 있는 폴더에 준비된 데이터를 써야 합니다. 이 두 단계는 원격 컴퓨팅에서 실행되며 각기 다른 컴퓨팅에서 실행할 수 있으므로, 작업 영역 내 데이터 저장소의 특정 위치에 대한 데이터 참조로 폴더 경로를 전달해야 합니다. **PipelineData** 개체는 중간 스토리지 위치에 사용되며 파이프라인 단계 간에 전달할 수 있는 특수한 종류의 데이터 참조입니다. 여기서는 PipelineData 개체를 만들어 첫 번째 단계의 출력/두 번째 단계의 입력으로 사용할 것입니다. 또한 데이터 참조를 통해 참조하는 데이터 저장소 위치에 코드가 액세스할 수 있도록 이 개체를 스크립트 인수로 전달해야 합니다.

In [ ]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 모델 폴더용 PipelineData(임시 데이터 참조) 만들기
prepped_data_folder = PipelineData("prepped_data_folder", datastore=ws.get_default_datastore())

# 1단계: 데이터 준비 스크립트 실행
train_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_diabetes.py",
                                arguments = ['--input-data', diabetes_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data_folder],
                                outputs=[prepped_data_folder],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# 2단계: 학습 스크립트 실행
register_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train_diabetes.py",
                                arguments = ['--training-folder', prepped_data_folder],
                                inputs=[prepped_data_folder],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

이제 정의한 단계에서 파이프라인을 작성하여 실험으로 실행할 준비가 되었습니다.

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# 파이프라인 생성
pipeline_steps = [train_step, register_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# 실험 작성 및 파이프라인 실행
experiment = Experiment(workspace=ws, name = 'mslearn-diabetes-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

파이프라인 실험이 실행되면 위젯에 해당 실험이 그래픽으로 표시됩니다. 페이지 오른쪽 위의 커널 표시기가 **&#9899;**에서 **&#9711;**로 바뀌면 코드 실행이 완료된 것입니다. [Azure Machine Learning Studio](https://ml.azure.com)의 **실험** 페이지에서 파이프라인 실행을 모니터링할 수도 있습니다.

파이프라인이 완료되고 나면 하위 실행에서 기록된 메트릭을 검사할 수 있습니다.

In [ ]:
for run in pipeline_run.get_children():
    print(run.name, ':')
    metrics = run.get_metrics()
    for metric_name in metrics:
        print('\t',metric_name, ":", metrics[metric_name])

파이프라인 실행이 정상적으로 완료되면 *학습 컨텍스트* 태그가 지정된 새 모델이 등록됩니다. 이 태그는 해당 모델이 파이프라인에서 학습되었음을 나타냅니다. 다음 코드를 사용하여 모델 등록 여부를 확인합니다.

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')